In [1]:
import pandas as pd
import numpy as np

In [80]:
def TopN_Arbitrage(date, starting_funds, N):
    curr = pd.read_csv("RprtRateXchg_20170930_20220930.csv")
    curr_latest = curr[curr["Record Date"] == date]
    curr_latest["Current Cash"] = starting_funds * curr_latest["Exchange Rate"]
    three = curr_latest[["Country - Currency Description",
                         "Exchange Rate", "Current Cash"]]
    cross1 = three.merge(three, how="cross")
    cross1["Currency Path"] = cross1["Country - Currency Description_x"] + \
        ", " + cross1["Country - Currency Description_y"]
    cross1 = cross1[cross1["Currency Path"] != (
        cross1["Country - Currency Description_x"] + ", " + cross1["Country - Currency Description_x"])]
    cross1["conversion"] = cross1["Exchange Rate_x"] / \
        cross1["Exchange Rate_y"]
    cross1["Current Cash2"] = cross1["Current Cash_x"] / cross1["conversion"]
    cross1 = cross1[cross1["Currency Path"] != (
        cross1["Country - Currency Description_x"] + ", " + cross1["Country - Currency Description_x"])]
    cross1 = cross1.rename(columns={'Country - Currency Description_x': 'First Country',
                                    'Exchange Rate_x': 'Exchange Rate_First',
                                    'Country - Currency Description_y': 'Second Country',
                                    'Exchange Rate_y': 'Exchange Rate_Second',
                                    'Current Cash_x': 'Cash After First Conversion',
                                    'Current Cash_y': 'Expected Cash Before Conversion',
                                    'Current Cash2': 'Actual Cash After Conversion'})
    cross1 = cross1[["First Country", "Second Country", "Currency Path",
                     "Cash After First Conversion",
                     "Exchange Rate_Second", "Expected Cash Before Conversion",
                     "Actual Cash After Conversion"]]
    cross1["Final Cash"] = (cross1["Actual Cash After Conversion"] /
                            cross1["Exchange Rate_Second"]) - starting_funds
    top_5 = cross1.sort_values(by=['Final Cash'], ascending=False)[0:N]
    return top_5[["Currency Path", "Final Cash"]]


top1 = TopN_Arbitrage('2022-06-30', 500000000, 1)
top1

/var/folders/47/6vn6cfqs6pv5sd36cd0n_sxr0000gn/T/ipykernel_14185/169427337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_latest["Current Cash"] = starting_funds * curr_latest["Exchange Rate"]


,Currency Path,Final Cash
27681,"Syria-Pound, Uruguay-Peso",1.192093e-07


In [48]:
curr = pd.read_csv("RprtRateXchg_20170930_20220930.csv")
curr["Record Date"][170]

'2022-06-30'

In [46]:
curr_latest = curr[curr["Record Date"] == "2022-09-30"]
curr_latest

,Record Date,Country,Currency,Country - Currency Description,Exchange Rate,Effective Date,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
0,2022-09-30,Afghanistan,Afghani,Afghanistan-Afghani,88.040,2022-09-30,1,2022,4,2022,3,9,30
1,2022-09-30,Albania,Lek,Albania-Lek,118.930,2022-09-30,2,2022,4,2022,3,9,30
2,2022-09-30,Algeria,Dinar,Algeria-Dinar,140.422,2022-09-30,3,2022,4,2022,3,9,30
3,2022-09-30,Angola,Kwanza,Angola-Kwanza,423.412,2022-09-30,4,2022,4,2022,3,9,30
4,2022-09-30,Antigua & Barbuda,E. Caribbean Dollar,Antigua & Barbuda-E. Caribbean Dollar,2.700,2022-09-30,5,2022,4,2022,3,9,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2022-09-30,Vietnam,Dong,Vietnam-Dong,23855.000,2022-09-30,164,2022,4,2022,3,9,30
164,2022-09-30,Western Samoa,Tala,Western Samoa-Tala,2.763,2022-09-30,165,2022,4,2022,3,9,30
165,2022-09-30,Yemen,Rial,Yemen-Rial,580.000,2022-09-30,166,2022,4,2022,3,9,30
166,2022-09-30,Zambia,New Kwacha,Zambia-New Kwacha,15.734,2022-09-30,167,2022,4,2022,3,9,30


In [11]:
starting_funds = 10000000
curr_latest["Current Cash"] = starting_funds * curr_latest["Exchange Rate"]
curr_latest[curr_latest["Currency"] == "Euro"]

/var/folders/47/6vn6cfqs6pv5sd36cd0n_sxr0000gn/T/ipykernel_14185/1102972224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_latest["Current Cash"] = starting_funds * curr_latest["Exchange Rate"]


,Record Date,Country,Currency,Country - Currency Description,Exchange Rate,Effective Date,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number,Current Cash
54,2022-09-30,Euro Zone,Euro,Euro Zone-Euro,1.026,2022-09-30,55,2022,4,2022,3,9,30,10260000.0


In [12]:
# Three most important columns that we care about!
three = curr_latest[["Country - Currency Description",
                     "Exchange Rate", "Current Cash"]]
three

,Country - Currency Description,Exchange Rate,Current Cash
0,Afghanistan-Afghani,88.040,8.804000e+08
1,Albania-Lek,118.930,1.189300e+09
2,Algeria-Dinar,140.422,1.404220e+09
3,Angola-Kwanza,423.412,4.234120e+09
4,Antigua & Barbuda-E. Caribbean Dollar,2.700,2.700000e+07
...,...,...,...
163,Vietnam-Dong,23855.000,2.385500e+11
164,Western Samoa-Tala,2.763,2.763000e+07
165,Yemen-Rial,580.000,5.800000e+09
166,Zambia-New Kwacha,15.734,1.573400e+08


In [21]:
cross1 = three.merge(three, how="cross")
cross1["Currency Path"] = cross1["Country - Currency Description_x"] + \
    ", " + cross1["Country - Currency Description_y"]
cross1 = cross1[cross1["Currency Path"] != (
    cross1["Country - Currency Description_x"] + ", " + cross1["Country - Currency Description_x"])]
cross1["conversion"] = cross1["Exchange Rate_x"] / cross1["Exchange Rate_y"]
cross1["Current Cash2"] = cross1["Current Cash_x"] / cross1["conversion"]
cross1 = cross1[cross1["Currency Path"] != (
    cross1["Country - Currency Description_x"] + ", " + cross1["Country - Currency Description_x"])]
cross1 = cross1.rename(columns={'Country - Currency Description_x': 'First Country', 
                                'Exchange Rate_x': 'Exchange Rate_First',
                                'Country - Currency Description_y': 'Second Country',
                                'Exchange Rate_y': 'Exchange Rate_Second',
                                'Current Cash_x': 'Cash After First Conversion',
                                'Current Cash_y': 'Expected Cash Before Conversion',
                                'Current Cash2': 'Actual Cash After Conversion'})
cross1 = cross1[["First Country", "Second Country", "Currency Path", 
                 "Cash After First Conversion",
                 "Exchange Rate_Second", "Expected Cash Before Conversion",
                 "Actual Cash After Conversion"]]
cross1

,First Country,Second Country,Currency Path,Cash After First Conversion,Exchange Rate_Second,Expected Cash Before Conversion,Actual Cash After Conversion
1,Afghanistan-Afghani,Albania-Lek,"Afghanistan-Afghani, Albania-Lek",8.804000e+08,118.930,1.189300e+09,1.189300e+09
2,Afghanistan-Afghani,Algeria-Dinar,"Afghanistan-Afghani, Algeria-Dinar",8.804000e+08,140.422,1.404220e+09,1.404220e+09
3,Afghanistan-Afghani,Angola-Kwanza,"Afghanistan-Afghani, Angola-Kwanza",8.804000e+08,423.412,4.234120e+09,4.234120e+09
4,Afghanistan-Afghani,Antigua & Barbuda-E. Caribbean Dollar,"Afghanistan-Afghani, Antigua & Barbuda-E. Cari...",8.804000e+08,2.700,2.700000e+07,2.700000e+07
5,Afghanistan-Afghani,Argentina-Peso,"Afghanistan-Afghani, Argentina-Peso",8.804000e+08,153.000,1.530000e+09,1.530000e+09
...,...,...,...,...,...,...,...
28218,Zimbabwe-RTGS,Venezuela-Fuerte (OLD),"Zimbabwe-RTGS, Venezuela-Fuerte (OLD)",6.063450e+09,248832.000,2.488320e+12,2.488320e+12
28219,Zimbabwe-RTGS,Vietnam-Dong,"Zimbabwe-RTGS, Vietnam-Dong",6.063450e+09,23855.000,2.385500e+11,2.385500e+11
28220,Zimbabwe-RTGS,Western Samoa-Tala,"Zimbabwe-RTGS, Western Samoa-Tala",6.063450e+09,2.763,2.763000e+07,2.763000e+07
28221,Zimbabwe-RTGS,Yemen-Rial,"Zimbabwe-RTGS, Yemen-Rial",6.063450e+09,580.000,5.800000e+09,5.800000e+09


In [23]:
cross1["Final Cash"] = (cross1["Actual Cash After Conversion"] /
                        cross1["Exchange Rate_Second"]) - starting_funds
cross1

,First Country,Second Country,Currency Path,Cash After First Conversion,Exchange Rate_Second,Expected Cash Before Conversion,Actual Cash After Conversion,Final Cash
1,Afghanistan-Afghani,Albania-Lek,"Afghanistan-Afghani, Albania-Lek",8.804000e+08,118.930,1.189300e+09,1.189300e+09,1.862645e-09
2,Afghanistan-Afghani,Algeria-Dinar,"Afghanistan-Afghani, Algeria-Dinar",8.804000e+08,140.422,1.404220e+09,1.404220e+09,0.000000e+00
3,Afghanistan-Afghani,Angola-Kwanza,"Afghanistan-Afghani, Angola-Kwanza",8.804000e+08,423.412,4.234120e+09,4.234120e+09,0.000000e+00
4,Afghanistan-Afghani,Antigua & Barbuda-E. Caribbean Dollar,"Afghanistan-Afghani, Antigua & Barbuda-E. Cari...",8.804000e+08,2.700,2.700000e+07,2.700000e+07,0.000000e+00
5,Afghanistan-Afghani,Argentina-Peso,"Afghanistan-Afghani, Argentina-Peso",8.804000e+08,153.000,1.530000e+09,1.530000e+09,0.000000e+00
...,...,...,...,...,...,...,...,...
28218,Zimbabwe-RTGS,Venezuela-Fuerte (OLD),"Zimbabwe-RTGS, Venezuela-Fuerte (OLD)",6.063450e+09,248832.000,2.488320e+12,2.488320e+12,0.000000e+00
28219,Zimbabwe-RTGS,Vietnam-Dong,"Zimbabwe-RTGS, Vietnam-Dong",6.063450e+09,23855.000,2.385500e+11,2.385500e+11,0.000000e+00
28220,Zimbabwe-RTGS,Western Samoa-Tala,"Zimbabwe-RTGS, Western Samoa-Tala",6.063450e+09,2.763,2.763000e+07,2.763000e+07,0.000000e+00
28221,Zimbabwe-RTGS,Yemen-Rial,"Zimbabwe-RTGS, Yemen-Rial",6.063450e+09,580.000,5.800000e+09,5.800000e+09,0.000000e+00


In [79]:
print(len(cross1[cross1["Final Cash"] > 1000]), len(cross1))
top = cross1.sort_values(by=['Final Cash'], ascending=False)[0:1]
top[["Currency Path", "Final Cash"]]

0 28056


,Currency Path,Final Cash
1,"Afghanistan-Afghani, Albania-Lek",1.862645e-09
